In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_mldata
from functools import reduce
mnist = fetch_mldata('MNIST original')


/home/chris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X,y = mnist["data"], mnist["target"]
#shuffle_index = np.random.permutation(60000)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]


In [13]:
def pix_split(rank):
    Y = np.zeros(rank+1)
    Y[rank] = 256
    for r in np.arange(rank):
        Y[r] = (255 // rank) * r
    return Y

def gen(rank):
    Y = pix_split(rank)
    A = np.array([[1,2],[0,1]])
    gen = [A]
    b = np.identity(2)
    b_ = np.identity(2)
    if rank == 2:
        gen.append(np.array([[1,0],[2,1]]))
    else:
        for r in np.arange(rank - 1):
            b = np.matmul(b,[[1,0],[2,1]])
            b_ = np.matmul(b_,[[1,0],[-2,1]])
            gen.append(np.matmul(np.matmul(b_,A),b))
    return gen

def sfe_rank(X,features,rank):
    g = gen(rank)
    ps = pix_split(rank)
    n= X.shape
#n = X_train.shape
#features = 100
    div = int(features / 4)
    k= int(np.floor(n[1] / div))
    l = k+1
    a, b = np.array([[k,l],[1,1]]), np.array([n[1],div])
    x,y = np.linalg.solve(a,b)
    x,y = int(np.rint(x)), int(np.rint(y))
    d = np.asarray([
  np.asarray(        
        np.append(
            np.asarray([
            reduce(
                lambda w,u: np.matmul(w,u),np.asarray(
                    [g[p] for z in X[j][k*i: k*(i+1)] for p in np.arange(rank + 1) if ps[p] <= z < ps[p+1] ]
                    )                                           
                    )
            for i in np.arange(x)
                        ]),
        np.asarray([
        reduce(
            lambda w,u: np.matmul(w,u),np.asarray(
                [g[p] for z in X[j][(x*k + l*i): (x*k + l * (i+1))]  for p in np.arange(rank + 1) if ps[p] <= z < ps[p+1] ]
                                                )
                )
        for i in np.arange(y)
        ])
        )
    )
for j in np.arange(n[0])
])
    return d

In [4]:
def sfe(X,features):
    n= X.shape
#n = X_train.shape
#features = 100
    div = int(features / 4)
    k= int(np.floor(n[1] / div))
    l = k+1
    a, b = np.array([[k,l],[1,1]]), np.array([n[1],div])
    x,y = np.linalg.solve(a,b)
    x,y = int(np.rint(x)), int(np.rint(y))
    d = np.asarray([
  np.asarray(        
        np.append(
            np.asarray([
            reduce(
                lambda w,u: np.matmul(w,u),[
                    [[1,2],[0,1]] if z <= 127 else [[1,0],[2,1]] for z in X[j][k*i: k*(i+1)] 
                                            ]
                    )
            for i in np.arange(x)
                        ]),
        np.asarray([
        reduce(
            lambda w,u: np.matmul(w,u),[
                [[1,2],[0,1]] if z <= 127 else [[1,0],[2,1]] for z in X[j][(x*k + l*i): (x*k + l * (i+1))] 
                                        ]
                )
        for i in np.arange(y)
        ])
        )
    )
for j in np.arange(n[0])
])
    return d

In [ ]:
def sfe3(X,features):
    g = [np.array([[1,4],[0,1]]), np.array([[1,2],[2,5]]), np.array([[1,0],[4,1]])]
    ps = pix_split(3)
    n= X.shape
#n = X_train.shape
#features = 100
    div = int(features / 4)
    k= int(np.floor(n[1] / div))
    l = k+1
    a, b = np.array([[k,l],[1,1]]), np.array([n[1],div])
    x,y = np.linalg.solve(a,b)
    x,y = int(np.rint(x)), int(np.rint(y))
    d = np.asarray([
  np.asarray(        
        np.append(
            np.asarray([
            reduce(
                lambda w,u: np.matmul(w,u),np.asarray(
                    [g[p] for z in X[j][k*i: k*(i+1)] for p in np.arange(3 + 1) if ps[p] <= z < ps[p+1] ]
                    )                                           
                    )
            for i in np.arange(x)
                        ]),
        np.asarray([
        reduce(
            lambda w,u: np.matmul(w,u),np.asarray(
                [g[p] for z in X[j][(x*k + l*i): (x*k + l * (i+1))]  for p in np.arange(3 + 1) if ps[p] <= z < ps[p+1] ]
                                                )
                )
        for i in np.arange(y)
        ])
        )
    )
for j in np.arange(n[0])
])
    return d

In [ ]:
features = 100 #needs to be divisible by 4
rank = 2
X_train_split_fe2 = sfe_rank(X_train,features)
X_test_split_fe2 = sfe_rank(X_test,features)

In [16]:
features = 100
rank = 2
test_wo_rank = sfe(X_train[0:1],features)
test_w_rank = sfe_rank(X_train[0:1], features,rank)

In [19]:
test_wo_rank[0], test_w_rank[0]

(array([     1,     62,      0,      1,      1,     62,      0,      1,
             1,     62,      0,      1,      1,     62,      0,      1,
            49,   2360,      6,    289,    353,     44,   3538,    441,
          3957,    884,  15922,   3557,   6781,  28170,    188,    781,
         40961, 419278,   1360,  13921,   8761, 142322,    396,   6433,
          9689, 195376,    686,  13833,   4069, 106466,    660,  17269,
         17665,  38254,  71536, 154913,  10645, 109084,    586,   6005,
          7129, 115828,    586,   9521, 195409,  48192,  46910,  11569,
         11129,  90868,  22652, 184953,   7697, 139814,    346,   6285,
          5325, 149624,    376,  10565,   8845,   4362,   1462,    721,
           337,     42,   4726,    589,      1,     62,      2,    125,
             1,     64,      0,      1,      1,     64,      0,      1,
             1,     64,      0,      1]),
 array([     1,     62,      0,      1,      1,     62,      0,      1,
             1,     62

In [ ]:
sfle_index = np.random.permutation(784)
pix_index = sfle_index[0:features]
pix_index.sort()

a = X_train.shape
b = X_test.shape

X_train_rand_feat = np.zeros((a[0],features))
X_test_rand_feat = np.zeros((b[0],features))
for j in np.arange(a[0]):
    for i in np.arange(features):
        X_train_rand_feat[j][i] = X_train[j][pix_index[i]]
for j in np.arange(b[0]):
    for i in np.arange(features):
        X_test_rand_feat[j][i] = X_test[j][pix_index[i]]

In [ ]:
data = X_train_split_fe2
scaler = StandardScaler()
scaler.fit(data)
X_scaled_train_split_fe = scaler.transform(data)

datatest = X_test_split_fe2
scaler = StandardScaler()
scaler.fit(datatest)
X_scaled_test_split_fe = scaler.transform(datatest)


In [ ]:
data = X_train_rand_feat
scaler = StandardScaler()
scaler.fit(data)
X_scaled_train_rand_feat = scaler.transform(data)

datatest = X_test_rand_feat
scaler = StandardScaler()
scaler.fit(datatest)
X_scaled_test_rand_feat = scaler.transform(datatest)

In [ ]:
X_test_split_fe3[1], X_scaled_test_split_fe[1]

In [ ]:
n_inputs = features
n_hidden1 = int(features / 2)
n_hidden2 = int(n_hidden1 / 2)
n_outputs = 10

In [ ]:
from tensorflow.contrib.layers import fully_connected

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope = "hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs", activation_fn = None)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()


n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(60000 // batch_size):
            shuffle_index = np.random.permutation(60000)
            batch_index = shuffle_index[0:batch_size]
            X_batch, y_batch = X_scaled_train_split_fe[batch_index], y_train[batch_index]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_scaled_test_split_fe,y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./f2_model_final.ckpt")

In [ ]:
def simplify(X):
    n,m = X.shape
    Y = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            if X[i,j] <= 127:
                Y[i,j] = 0
            else:
                Y[i,j] = 255
    return Y

In [ ]:
data = X_train_rand_feat
scaler = StandardScaler()
scaler.fit(data)
X_scaled_train_rand_feat = scaler.transform(data)

datatest = X_test_rand_feat
scaler = StandardScaler()
scaler.fit(datatest)
X_scaled_test_rand_feat = scaler.transform(datatest)

In [ ]:
X_scaled_train_rand_feat[0]

In [ ]:
X_train_simp[1], X_test_simp[1]

In [ ]:
some_digit = X_test_rand_feat[1001]
some_digit_image = some_digit.reshape(28, 28)
plt.imshow(some_digit_image, cmap = matplotlib.cm.binary,
interpolation="nearest")
plt.axis("off")
plt.show()